In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# <span style="color:red;"><center>TD_03_2 - RSA 1/2 mise en place des outils</center></span>

Le chiffrement RSA est nommé d'après les initiales de ses trois inventeurs Ronald Rivest, Adi Shamir et Leonard Adleman. C'est une méthode de chiffrement asymétrique inventée en 1977 et encore utilisée aujourd'hui pour le commerce électronique ou encore l'échange de données confidentielles sur internet.


Le TD se découpe en deux parties, une première pour mettre en places les différents outils nécessaires puis une seconde sur l'algorithme RSA spécifiquement.

## Exercice 1. Exponentiation rapide modulaire

1. Rappeler ici la fonction sur laquelle nous avons déjà travaillé qui prend trois arguments x, n et m et qui renvoie le résultat de $x^n (mod\ m)$. Cette fonction devra être récursive (pour s'entrainer) et utiliser $x^0 = 1$ et $x^n = ((x^2\ mod\ m)^{n//2})\ mod\ m$ si n est pair et $x^n = ((x\ mod\ m) \times (x^2\ mod\ m)^{n//2})\ mod\ m$ si n est impair.

In [4]:
# question 1.
def puissance_mod(x,n,m):
    """Renvoie x puissance n modulé par m.
    x est un int ou un float
    n est un int > 0
    m est un int"""
    assert type(x) == float or type(x) == int
    assert type(n) == int and n >= 0
    assert type(m) == int
    if n == 0:
        return 1
    else:
        return ((x%m)**(n%2) * puissance_mod(x*x,n//2,m))%m

puissance_mod(2,10,100)

24

## Exercice 2. Décomposer un texte en une liste de valeurs

Nous avons déjà travaillé sur les fonctions **chr** et **ord** 


On souhaite décomposer un texte sous forme d’une liste de valeurs numériques sur lesquelles travailler. On utilisera les valeurs de la table ASCII accessibles grâce à la fonction python ord(car) qui prend un seul caractère comme paramètre.

1. Ecrire une fonction **code(texte)** qui prend en entrée une chaine de caractères texte et qui renvoie une liste correspondant à la valeur de chaque caractère dans la table ASCII.
       
2. Ecrire une fonction **decode(tab)** qui effectue l’opération inverse. On pourra utiliser la fonction python chr(i) qui prend comme paramètre un entier et effectue l’opération inverse de ord(car).
       
3. Tester les fonctions obtenues à l’aide de phrases de votre choix

In [5]:
# Question 1.
def code(txt):
    """Renvoie une liste correspondant à la valeur de chaque caractère dans la table ASCII.
    txt est un str"""
    assert type(txt) == str
    output = []
    for cara in txt:
        output.append(ord(cara))
    return output

In [6]:
# Question 2.
def decode(tab):
    """Renvoie une chaîne de caractères correspondant aux valeurs de tab dans la table ASCII.
    tab est un list"""
    assert type(tab) == list
    output = ""
    for val in tab:
        output += chr(val)
    return output

In [7]:
# Question 3.
s = code("Saumon de norvège.")
print(s)
print(decode(s), "\n")

b = code("Une baguette pas trop cuite s'il vous plaît.")
print(b)
print(decode(b))

[83, 97, 117, 109, 111, 110, 32, 100, 101, 32, 110, 111, 114, 118, 232, 103, 101, 46]
Saumon de norvège. 

[85, 110, 101, 32, 98, 97, 103, 117, 101, 116, 116, 101, 32, 112, 97, 115, 32, 116, 114, 111, 112, 32, 99, 117, 105, 116, 101, 32, 115, 39, 105, 108, 32, 118, 111, 117, 115, 32, 112, 108, 97, 238, 116, 46]
Une baguette pas trop cuite s'il vous plaît.


## Exercice 3. Découpage du texte par blocs

Pour éviter une attaque statistique, il faut découper le message original en blocs. Ici, on se limites aux caractères de la table ASCII, chaque caractère se code donc sur trois chiffres (le caractère 65 sera donc codé 065). On collera ensuite tous les chiffres obtenus dans l’ordre puis on découpera par blocs de taille fixée (4 par exemple). On comblera le dernier bloc en ajoutant des 0.

exemple:
« Bonjour » donnerait [66,111,110,106,111,117,114]
en rajoutant les zéros pour coder chaque caractère sur trois chiffres [066, 111, 110, 106, 111, 117, 114]
puis on regroupe tout 066111110106111117114
Enfin on découpe par blocs de 4 et on complète le dernier bloc par des 0. [0661, 1111, 0106, 1111, 1711, 4000].


Il faudra passer par les chaines de caractères pour ne pas perdre les zéros en début de bloc. La liste finale renvoyée ne devra contenir que des entiers car nous réaliseront le chiffrement sur ces valeurs.

1. Ecrire une fonction **trois** qui prend comme argument un caractère et qui renvoie une chaine de trois caractères (exactement) correspondants au codage sur trois chiffres du caractère de départ.

2. Ecrire une fonction **regroupe** qui prend comme paramètre une chaine de caractères et qui renvoie la concaténation des différents blocs de 3 caractères obtenus.

3. Ecrire une fonction **decoupe** qui prend en paramètre une chaine de caractères et un entier n et qui renvoie une liste de chaines de n caractères correspondant au découpage par blocs après codage ascii de chaque caractère. Elle devra aussi compléter le dernier bloc avec des 0.

4. Proposer une fonction **valeurs** qui prend une chaine de caractères en entrée et qui renvoie la liste des entiers obtenus après découpage.

5. Proposer une fonction **texte** qui effectue l’opération inverse (vous pouvez proposer toutes les fonctions intermédiaires de votre choix)

In [8]:
# Question 1.
def trois(cara):
    """Renvoie une chaine de trois caractères correspondants au codage sur trois chiffres du caractère de départ.
    cara est un str de len = 1"""
    assert type(cara) == str and len(cara) == 1
    output = str(ord(cara))
    while len(output) < 3:
        output = "0"+output
    return output

trois("a")

'097'

In [9]:
# Question 2.
def regroupe(txt):
    """Renvoie la concaténation des différents blocs de 3 caractères obtenus.
    txt est un str"""
    assert type(txt) == str
    output = []
    for cara in txt:
        output.append(trois(cara))
    return "".join(output)

regroupe("Bonjour")

'066111110106111117114'

In [10]:
# Question 3.
def decoupe(txt, n):
    """Renvoie une liste de chaines de n caractères correspondant au découpage par blocs après codage ascii de chaque caractère. Le dernier bloc est complété avec des 0.
    txt est un str
    n est un int"""
    assert type(txt) == str
    assert type(n) == int
    vals = regroupe(txt)
    output = []
    for i in range(0, len(vals), n):
        output.append(vals[i:i+n])
    while len(output[-1]) < n:
        output[-1] += "0"
    return output

decoupe("Bonjour", 4)

['0661', '1111', '0106', '1111', '1711', '4000']

In [11]:
# Question 4.
def valeurs(txt, n):
    """Renvoie la liste des entiers obtenus après découpage."""
    return list(map(int, decoupe(txt, n)))

valeurs("Bonjour", 4)

[661, 1111, 106, 1111, 1711, 4000]

In [12]:
# Question 5.
def texte(tab, n):
    """Décode un tableau d'entiers.
    tab est un list
    n est un int"""
    assert type(tab) == list
    assert type(n) == int
    tab = list(map(str, tab))
    for i in range(len(tab)-1):
        while len(tab[i]) < n:
            tab[i] = "0"+tab[i]
    regrouped = "".join(tab)
    output = []
    for i in range(0, len(regrouped)-1, 3):
        output.append(regrouped[i:i+3])
    output = list(map(int, output))
    while output[-1] == 0:
        output.pop()
    for i in range(len(output)):
        output[i] = chr(output[i])
    return "".join(output)

texte([661, 1111, 106, 1111, 1711, 4000], 4)

'Bonjour'

## Exercice 4. Euclide et PGCD

1. Ecrire une fonction **choix_pq** qui renvoie deux nombres premiers choisis aléatoirement (parmi ceux générés au TD précédent par exemple).

2. Ecrire une fonction **Euclide** qui prend comme arguments deux entiers a et b et qui renvoie le quotient et le reste de la division Euclidienne de a par b.

3. Ecrire une fonction **PGCD** qui prend comme paramètres deux entiers a et b et qui renvoie leur PGCD (plus grand diviseur commun). Pour cela, on remarquera que PGCD(a, b) = PGCD(b, a%b)

4. Ecrire une fonction **Bezout** qui prend comme paramètres deux entiers a et b et qui renvoie les coefficients de l’égalité de Bézout c’est à dire un couple (u, v) tel que a*u + b*v = PGCD(a,b). Cette fonction peut aussi renvoyer le PGCD(a,b)
Pour cela on initialise u1 à 1, v1 à 0, u2 à 0 et v2 à 1
Puis on réalise la boucle :
  Soit q et r le quotient et le reste de la division euclidienne de a par b, on a donc:
  
  
  $a = q \times b + r$ soit $u_1 \times a + v_1 \times b = q \times (u_2 \times a + v2 \times b) + r$ et donc $(u_1 - q \times u_2) \times a + (v_1 - q \times v_2) \times b = r$
  
  
On va ensuite calculer le quotient et le reste de la division euclidienne de $(u_2 \times a + v_2 \times b)$ par r et ainsi de suite jusqu’à obtenir un reste nul.

Le dernier reste non nul obtenu est le PGCD(a,b), et les derniers coefficients $u_1 - q \times u_2$ et $v_1 - q \times v_2$ sont les deux coefficients de l’égalité de Bézout recherchés.

In [19]:
# Question 1.
def initialise_crible(N):
    assert type(N) == int
    L = [True]*(N+1)
    L[0] = False
    L[1] = False
    return L
def barre_multiple(L, n):
    assert type(L) == list
    assert type(n) == int
    for i in range(n+1, len(L)):
        if i%n == 0:
            L[i] = False
    return L
def crible(N):
    assert type(N) == int
    L = initialise_crible(N)
    for i in range(2, len(L)):
        L = barre_multiple(L, i)
    return L
def liste_premier(N):
    assert type(N) == int
    L = crible(N)
    premier = []
    for i in range(2, len(L)):
        if L[i]:
            premier.append(i)
    return premier

from random import choice
def choix_pq(premiers):
    """premiers doit être une liste de nombres premiers. La fonction renvoie 2 éléments aléatoires de cette liste."""
    return choice(premiers), choice(premiers)

choix_pq(liste_premier(50))

(47, 43)

In [20]:
# Question 2.
def Euclide(a, b):
    """Renvoie le quotient et le reste de la division euclidienne de a par b."""
    return a//b, a%b

In [42]:
# Question 3.
def PGCD(a, b):
    """Renvoie le plus grand diviseur commun de a et b."""
    if a <= b:
        PGCD(b, a)
    if a%b == 0:
        return b
    else:
        return PGCD(b, a%b)

PGCD(101, 97)
PGCD(1024, 50)
PGCD(3, 9)

1

2

3

In [64]:
# Question 4.
def bezout(a, b, u1=1, u2=0, v1=0, v2=1):
    """Renvoie les coefficients de l'égalité de Bezout."""
    if a <= b:
        bezout(b, a)
    elif a%b == 0:
        return u2, v2
    else:
        return bezout(b, a%b, u2, u1-a//b*u2, v2, v1-a//b*v2)

bezout(101,97)

(-24, 25)

Tous les outils sont maintenant prêts, nous allons pouvoir commencer à explorer l'algorithme RSA!